## Import necessary Libraries

In [8]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import folium # plotting library


## Define Foursquare Credentials and Version 

In [4]:
CLIENT_ID = 'S3N1EOXFANGIHWZ0NPJ21OZJ0A2INSCC3V4ZGLHX4JWOWEE3' # your Foursquare ID
CLIENT_SECRET = 'C3O1FJ5QSKCR4WDL0UI0H5ZWVXHYTT5ZDJDE0EW4ISWRXQG5' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: S3N1EOXFANGIHWZ0NPJ21OZJ0A2INSCC3V4ZGLHX4JWOWEE3
CLIENT_SECRET:C3O1FJ5QSKCR4WDL0UI0H5ZWVXHYTT5ZDJDE0EW4ISWRXQG5


# 1. Search for a specific venue category

In [21]:
address = '102 North End Ave, New York, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7149555 -74.0153365


# search query

In [18]:
search_query = 'Italian'
radius = 500
print(search_query + ' .... OK!')

Italian .... OK!


In [24]:
#Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=S3N1EOXFANGIHWZ0NPJ21OZJ0A2INSCC3V4ZGLHX4JWOWEE3&client_secret=C3O1FJ5QSKCR4WDL0UI0H5ZWVXHYTT5ZDJDE0EW4ISWRXQG5&ll=40.7149555,-74.0153365&v=20180604&query=Italian&radius=500&limit=30'

In [25]:
#Send the GET Request and examine the results
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f24366ffe1f6657f63766a9'},
 'response': {'venues': [{'id': '4fa862b3e4b0ebff2f749f06',
    'name': "Harry's Italian Pizza Bar",
    'location': {'address': '225 Murray St',
     'lat': 40.71521779064671,
     'lng': -74.01473940209351,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.71521779064671,
       'lng': -74.01473940209351},
      {'label': 'entrance', 'lat': 40.715361, 'lng': -74.014975}],
     'distance': 58,
     'postalCode': '10282',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['225 Murray St',
      'New York, NY 10282',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1ca941735',
      'name': 'Pizza Place',
      'pluralName': 'Pizza Places',
      'shortName': 'Pizza',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-15962092

In [12]:
# assign relevant part of JSON to venues
venues = results['response']['venue']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
print(dataframe.head())

                         id                       name  \
0  4fa862b3e4b0ebff2f749f06  Harry's Italian Pizza Bar   

                                        canonicalUrl  \
0  https://foursquare.com/v/harrys-italian-pizza-...   

                                          categories  verified  \
0  [{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...     False   

                        url  hasMenu  dislike     ok  rating  ...  \
0  http://harrysitalian.com     True    False  False     6.9  ...   

                       bestPhoto.prefix  \
0  https://fastly.4sqi.net/img/general/   

                                   bestPhoto.suffix  bestPhoto.width  \
0  /ya1iQFI7pLjuIJp1PGDKlrZS3OJdHCF7tpILMmjv_2w.jpg              480   

   bestPhoto.height bestPhoto.visibility colors.highlightColor.photoId  \
0               640               public      4fad980de4b091b4626c3633   

  colors.highlightColor.value colors.highlightTextColor.photoId  \
0                   -13619152          4fad980de4b

In [13]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

print(dataframe_filtered)

                        name   categories        address        lat  \
0  Harry's Italian Pizza Bar  Pizza Place  225 Murray St  40.715218   

         lng                                     labeledLatLngs postalCode  \
0 -74.014739  [{'label': 'display', 'lat': 40.71521779064671...      10282   

   cc      city state        country  \
0  US  New York    NY  United States   

                                    formattedAddress                        id  
0  [225 Murray St, New York, NY 10282, United Sta...  4fa862b3e4b0ebff2f749f06  


In [16]:
dataframe_filtered.name
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

# 2  Explore a Given Venue 

In [26]:
venue_id = '4fa862b3e4b0ebff2f749f06' # ID of Harry's Italian Pizza Bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4fa862b3e4b0ebff2f749f06?client_id=S3N1EOXFANGIHWZ0NPJ21OZJ0A2INSCC3V4ZGLHX4JWOWEE3&client_secret=C3O1FJ5QSKCR4WDL0UI0H5ZWVXHYTT5ZDJDE0EW4ISWRXQG5&v=20180604'

In [27]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'hasMenu', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'menu', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'seasonalHours', 'defaultHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4fa862b3e4b0ebff2f749f06',
 'name': "Harry's Italian Pizza Bar",
 'contact': {'phone': '2126081007', 'formattedPhone': '(212) 608-1007'},
 'location': {'address': '225 Murray St',
  'lat': 40.71521779064671,
  'lng': -74.01473940209351,
  'labeledLatLngs': [{'label': 'display',
    'lat': 40.71521779064671,
    'lng': -74.01473940209351},
   {'label': 'entrance', 'lat': 40.715361, 'lng': -74.014975}],
  'postalCode': '10282',
  'cc': 'US',
  'city': 'New York',
  'state': 'NY',
  'country': 'United States',
  'formattedAddress': ['225 Murray St',
   'New York, NY 10282',
   'United States']},
 'canonicalUrl': 'https://foursquare.com/v/harrys-italian-pizza-bar/4fa862b3e4b0ebff2f749f06',
 'categories': [{'id': '4bf58dd8d48988d1ca941735',
   'name': 'Pizza Place',
   'pluralName': 'Pizza Places',
   'shortName': 'Pizza',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
    'suffix': '.png'},
   'primary': True},
  {'id': '4bf58dd8d48988d110941735',
   'n

## B. Get the venue's overall rating

In [30]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

6.9


In [31]:
# this venue is not very good lets check another rating of nearest restraurant
venue_id = '4f3232e219836c91c7bfde94' # ID of Conca Cucina Italian Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [32]:
# check third restraurant
venue_id = '3fd66200f964a520f4e41ee3' # ID of Ecco
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.3


## C. Get the number of tips

In [33]:
result['response']['venue']['tips']['count']

19

## D. Get the venue's tips

In [34]:
#Create URL and send GET request. Make sure to set limit to get all tips
## Ecco Tips
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f2439912c47b45976ca84fb'},
 'response': {'tips': {'count': 19,
   'items': [{'id': '5ab1cb46c9a517174651d3fe',
     'createdAt': 1521601350,
     'text': 'A+ Italian food! Trust me on this: my mom’s side of the family is 100% Italian. I was born and bred to know good pasta when I see it, and Ecco is one of my all-time NYC favorites',
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/5ab1cb46c9a517174651d3fe',
     'lang': 'en',
     'likes': {'count': 0, 'groups': []},
     'logView': True,
     'agreeCount': 4,
     'disagreeCount': 0,
     'todo': {'count': 0},
     'user': {'id': '484542633',
      'firstName': 'Nick',
      'lastName': 'E',
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/484542633_unymNUmw_FdPs3GjXHujmHcYnN4hf8kEPADlOZuIrdcdm97VX3tFqL7fFNMNA_8Gl9NlU1GYg.jpg'}},
     'authorInteractionType': 'liked'}]}}}

# 3. Explore a location

In [35]:
latitude = 40.715337
longitude = -74.008848
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=S3N1EOXFANGIHWZ0NPJ21OZJ0A2INSCC3V4ZGLHX4JWOWEE3&client_secret=C3O1FJ5QSKCR4WDL0UI0H5ZWVXHYTT5ZDJDE0EW4ISWRXQG5&ll=40.715337,-74.008848&v=20180604&radius=500&limit=30'

In [36]:
import requests

In [37]:
results = requests.get(url).json()
'There are {} around Ecco restaurant.'.format(len(results['response']['groups'][0]['items']))

'There are 30 around Ecco restaurant.'

In [38]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '5d5f24ec09484500079aee00',
  'name': 'Los Tacos No. 1',
  'location': {'address': '136 Church St',
   'lat': 40.714267,
   'lng': -74.008756,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.714267,
     'lng': -74.008756}],
   'distance': 119,
   'postalCode': '10007',
   'cc': 'US',
   'city': 'New York',
   'state': 'NY',
   'country': 'United States',
   'formattedAddress': ['136 Church St',
    'New York, NY 10007',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d151941735',
    'name': 'Taco Place',
    'pluralName': 'Taco Places',
    'shortName': 'Tacos',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/taco_',
     'suffix': '.png'},
    'primary': True}],
  'delivery': {'id': '2180700',
   'url': 'https://www.seamless.com/menu/los-tacos-no-1-tribeca-136-church-st-new-york/21

In [39]:
#Process JSON and convert it to a clean dataframe
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Los Tacos No. 1,Taco Place,136 Church St,40.714267,-74.008756,"[{'label': 'display', 'lat': 40.714267, 'lng':...",119,10007,US,New York,NY,United States,"[136 Church St, New York, NY 10007, United Sta...",NaN,NaN,5d5f24ec09484500079aee00
1,Korin,Furniture / Home Store,57 Warren St,40.714824,-74.009404,"[{'label': 'display', 'lat': 40.71482437714839...",73,10007,US,New York,NY,United States,"[57 Warren St (Church St), New York, NY 10007,...",Church St,Tribeca,4af5d65ff964a52091fd21e3
2,Takahachi Bakery,Bakery,25 Murray St,40.713653,-74.008804,"[{'label': 'display', 'lat': 40.71365284530189...",187,10007,US,New York,NY,United States,"[25 Murray St (at Church St), New York, NY 100...",at Church St,NaN,4c154c9a77cea593c401d260
3,Juice Press,Vegetarian / Vegan Restaurant,83 Murray St,40.714788,-74.011132,"[{'label': 'display', 'lat': 40.71478769908051...",202,10007,US,New York,NY,United States,[83 Murray St (btwn Greenwich St & W Broadway)...,btwn Greenwich St & W Broadway,NaN,54148bc6498ea7bb8c05b70a
4,Takahachi,Sushi Restaurant,145 Duane St,40.716526,-74.008101,"[{'label': 'display', 'lat': 40.71652647412374...",146,10013,US,New York,NY,United States,"[145 Duane St (btwn W Broadway & Church St), N...",btwn W Broadway & Church St,NaN,4a8f2f39f964a520471420e3
5,Chambers Street Wines,Wine Shop,148 Chambers St,40.715773,-74.009718,"[{'label': 'display', 'lat': 40.71577306392837...",88,10007,US,New York,NY,United States,[148 Chambers St (btwn West Broadway & Hudson ...,btwn West Broadway & Hudson St,NaN,4adcf23cf964a520cc6221e3
6,Nish Nūsh,Falafel Restaurant,88 Reade St,40.715537,-74.007725,"[{'label': 'display', 'lat': 40.71553710116416...",97,10013,US,New York,NY,United States,"[88 Reade St (at Church St), New York, NY 1001...",at Church St,NaN,50ba9119e4b071a4bae6dc10
7,Whole Foods Market,Grocery Store,270 Greenwich Street,40.715579,-74.011368,"[{'label': 'display', 'lat': 40.71557915542060...",214,10007,US,New York,NY,United States,"[270 Greenwich Street (at Warren St), New York...",at Warren St,Tribeca,49bc3b0af964a52020541fe3
8,Weather Up,Cocktail Bar,159 Duane St,40.716741,-74.008666,"[{'label': 'display', 'lat': 40.71674084163369...",157,10013,US,New York,NY,United States,"[159 Duane St (btwn Hudson St. & W Broadway), ...",btwn Hudson St. & W Broadway,NaN,4cd89eeb6e8b5941660c64d2
9,Lekka Burger,Burger Joint,81 Warren St,40.715246,-74.010559,"[{'label': 'display', 'lat': 40.715246, 'lng':...",144,10007,US,New York,NY,United States,"[81 Warren St (btw Greenwich & West Broadway),...",btw Greenwich & West Broadway,NaN,5dc6f6a5ea8dfb00080f6faa


In [40]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map